In [5]:
import tensorflow as tf
from tensorflow import keras
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [8]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [9]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [11]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [12]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [13]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

In [15]:
import numpy as np

In [20]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [28]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [29]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [31]:
np.random.seed(42)
tf.random.set_seed(42)

In [30]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [32]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [33]:
dataset = dataset.prefetch(1)

In [34]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


In [36]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10)

Epoch 1/10
31370/31370 [==============================] - 11946s 381ms/step - loss: 1.4661
Epoch 2/10
31370/31370 [==============================] - 8210s 262ms/step - loss: 1.3634
Epoch 3/10
31370/31370 [==============================] - 7968s 254ms/step - loss: 1.3437
Epoch 4/10
31370/31370 [==============================] - 8707s 278ms/step - loss: 1.3326
Epoch 5/10
31370/31370 [==============================] - 9842s 314ms/step - loss: 1.3253
Epoch 6/10
31370/31370 [==============================] - 11645s 371ms/step - loss: 1.3200
Epoch 7/10
31370/31370 [==============================] - 10854s 346ms/step - loss: 1.3157
Epoch 8/10
31370/31370 [==============================] - 9241s 295ms/step - loss: 1.3128
Epoch 9/10
31370/31370 [==============================] - 8246s 263ms/step - loss: 1.3104
Epoch 10/10
31370/31370 [==============================] - 11798s 376ms/step - loss: 1.3084


In [39]:
def preprocess(texts):
    X=np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(X,max_id)

In [43]:
def next_char(text, temperature=1):
    X_new=preprocess([text])
    y_proba=model.predict(X_new)[0,-1:,:]
    rescaled_logits=tf.math.log(y_proba)/temperature
    char_id=tf.random.categorical(rescaled_logits, num_samples=1)+1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text,n_chars=1000,temperature=1):
    for _ in range(n_chars):
        text+=next_char(text,temperature)
    return text

In [40]:
print(complete_text("w", temperature=1))

will the budy sister.

menenius:
what! where shall'


In [46]:
#### print(complete_text("a", temperature=1))

# all as i will part
# 'tis slrums from my best begun: with them am
# to the anselves, what the elder of double comes are their
# touches deliver to a chore.




print('''first citizen:
o madam, to the other baste. what's the enough?

treasure:
why, would i know 'tis thus in your counsel in the storing perhopies;
for when my advised woo i' the remonaste,
we hear the better lunators, which will charge her poor baws,
that be so says shall be more swork her soblicient more.

hortensio:
and say you give me but?

katharina:
o belly, they was no ready stay to save
the scold me the ale.
who was her? why, malicious tranio, since with the swer to beaute.

menenius:
belly the matter, he they are the sweal us curse
and fair the store--
for this men are be so,
sirrah you this godss with it; and that the coat.

first citizen:
all, knock leave thee, sir; and them esteeds,
as one of friend, in perite is girl! this idle from 
menenius:
what, this accised upon you. where this gremio.
to in the rail. this rogue stribe.
it i''')

first citizen:
o madam, to the other baste. what's the enough?

treasure:
why, would i know 'tis thus in your counsel in the storing perhopies;
for when my advised woo i' the remonaste,
we hear the better lunators, which will charge her poor baws,
that be so says shall be more swork her soblicient more.

hortensio:
and say you give me but?

katharina:
o belly, they was no ready stay to save
the scold me the ale.
who was her? why, malicious tranio, since with the swer to beaute.

menenius:
belly the matter, he they are the sweal us curse
and fair the store--
for this men are be so,
sirrah you this godss with it; and that the coat.

first citizen:
all, knock leave thee, sir; and them esteeds,
as one of friend, in perite is girl! this idle from 
menenius:
what, this accised upon you. where this gremio.
to in the rail. this rogue stribe.
it i


In [45]:
from random import randint
letters='qwertyuiopasdfghjklzxcvbnm'
index=randint(0,25)

print(complete_text(letters[index], temperature=1))

wrong, the city.

hortensio:
give himself, receive the rightal blow the rest,
they burn users, that must tell this give be honour
and me all particially as for it;
which gives a suitor heard withal instrued
and make god pirped in althe takes oc very deeds.

painio:
alive to common; even are away, i may leave her
balls pass, since i proceed and live it find for
the stolence hath thus flound the state,
but knock you faol, was more a counsel for the good the belly
would be loud and so.
behive you what you will not hake? you danciy perhaps of me;
he'll part the worsporants ere it?

maniantio:
become it that shall socient the good master,
this good fyceinor, well
his faults, i have her colding me not in my tears,
and do i will content to be marcived by a fair in hell:
if i proceed us belly once.

bianca:
who shall be countent to her way to leave not
his own prince well; even be not rates, i did no foll
if you plead boar beneyous the ready us words:
why ched you state, the opentastal pains a

In [47]:
!mkdir -p saved_model
model.save('saved_model/my_model') 

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model/assets
